In [1]:
import arcpy

Here, we'll look at how to use arcpy to make maps using a project that already has a layout with a map series setup.

https://pro.arcgis.com/en/pro-app/arcpy/mapping/mapseries-class.htm

https://pro.arcgis.com/en/pro-app/help/layouts/map-series.htm



In [14]:
# Load the project
project_path = r".\Beginner_Week_3.aprx"
project = arcpy.mp.ArcGISProject(project_path)

layouts = project.listLayouts()

In [15]:
# Get the layout with a map series
for layout in layouts:
    if not layout.mapSeries is None:
        print(layout.mapSeries.enabled)

True


In [16]:
layout

Now we'll have to fix some data sources (because I didn't set up my ArcGIS Pro Project very well)

In [17]:
# check for broken layers (layers with no source)

for layer in layout.mapSeries.mapFrame.map.listLayers():
    if layer.name in ['Highways_Intersect','Counties']:
        print(layer.name, layer.dataSource, layer.isBroken)

Highways_Intersect C:\Desktop\GIS_STUFF\Workshops\Workshop_Beginner_3\Week_2_Data.gdb\Highways_Intersect True
Counties C:\Desktop\GIS_STUFF\Workshops\Workshop_Beginner_3\Week_2_Data.gdb\Counties True


In [26]:
# reset the source for broken layers

for layer in layout.mapSeries.mapFrame.map.listLayers():
    if layer.name in ['Highways_Intersect','Counties']:
        layer.updateConnectionProperties(r'C:\Desktop\GIS_STUFF\Workshops\Workshop_Beginner_3','.')
        print(layer.name, layer.dataSource, layer.isBroken)
project.save()

Highways_Intersect C:\Users\DPCN\Documents\GitHub\Workshop_Beginner_3\Week_2_Data.gdb\Highways_Intersect False
Counties C:\Users\DPCN\Documents\GitHub\Workshop_Beginner_3\Week_2_Data.gdb\Counties False


Now we'll find the title so we can change the map title for the page we want to export.

In [20]:
print(layout.listElements())
title = layout.listElements()[0]

[<arcpy._mp.TextElement object at 0x00000000132E4828>, <arcpy._mp.TextElement object at 0x00000000132E4CC0>, <arcpy._mp.MapFrame object at 0x00000000132E4710>]


Now we can use the pages in our map series to get the page for a specific county.

In [27]:
ms = layout.mapSeries

In [28]:
ms.currentPageNumber

0

In [29]:
county = 'Marin County'

In [40]:
ms.currentPageNumber =  ms.getPageNumberFromName(county)

And we can filter the roads layer so we only see roads in the county we want

In [19]:
for layer in layout.mapSeries.mapFrame.map.listLayers():
    if layer.name == 'Highways_Intersect':
        roads_layer = layer

In [31]:
roads_layer.definitionQuery = "NAMELSAD = '{}'".format(county)
title.text = county


In [41]:
ms.exportToPDF('./{}.pdf'.format(county), 'CURRENT')

'./Marin County.pdf'